In [34]:
file_name = '/Users/teraimao/experiment/work/13_14_15_K,N,S,Y,F_work/analysis/ana_gbsw_in/rmsd/rmsd_backbone_cb_combined.dat'

In [35]:
import numpy as np

values = []
with open(file_name, 'r') as f:
    lines = f.readlines()
    for line in lines:
        if len(line.split()) >= 2:  # 3列以上あるか確認
            try:
                num = float(line.split()[2])  # 数値に変換
                values.append(num)  # 数値としてリストに追加
            except ValueError:
                pass  # 変換できない場合はスキップ
           
        
#values =[float(line.split()[2]) for line in lines if len(line.split()) >=2 ]
#average = sum(values) / len(values)



In [36]:
sub_set = values[400:500]
average = sum(sub_set)/len(sub_set)
print(average)

0.7411399999999998


In [4]:
sub_set = values[:100]
average = sum(sub_set)/len(sub_set)
print(average)

0.91


In [5]:
sub_set = values[:500]
average = sum(sub_set)/len(sub_set)
print(average)

0.844


In [65]:
import os

def calculate_surf_average(value):
    column_indices = [1, 2, 3, 4]  # 2列目、3列目、4列目、5列目のインデックス
    time_index = 0  # 計算時間のインデックス
    peptide_positions = ["out"]  # 各位置で処理
    directory = "surf_area"  # hbond専用ディレクトリ名

    try:
        for position in peptide_positions:
            dir_path = f"/Users/teraimao/experiment/work/{value}_work/analysis/ana_gbsw_{position}/{directory}"

            # ディレクトリの存在確認
            if not os.path.isdir(dir_path):
                print(f"Directory not found: {dir_path}")
                continue

            # ディレクトリ内のファイルをリストアップ
            data_files = [f for f in os.listdir(dir_path) if os.path.isfile(os.path.join(dir_path, f))]
            if not data_files:
                print(f"No files found in directory: {dir_path}")
                continue

            # 最初のファイルを選択（複数ファイルがある場合は要確認）
            file_path = os.path.join(dir_path, data_files[0])
            column_values = {index: [] for index in column_indices}

            with open(file_path, 'r') as file:
                next(file)  # ヘッダーをスキップ
                for line in file:
                    if not line.strip():  # 空行をスキップ
                        continue
                    columns = line.split()
                    time = float(columns[time_index])  # 時間列を取得
                    if time >= 40.0:  # 時間が40以上の場合のみ値を取得
                        for index in column_indices:
                            column_values[index].append(float(columns[index]))

            # 各列の平均を計算
            averages = {
                index: (sum(column_values[index]) / len(column_values[index])) if column_values[index] else 0.0
                for index in column_indices
            }

            # データがなかった場合の処理
            if not any(column_values.values()):
                print(f"No valid hbond values found in {file_path} for averaging.")
                continue

            # "average" ディレクトリを作成
            average_dir_path = os.path.join(dir_path, "average")
            os.makedirs(average_dir_path, exist_ok=True)

            # 結果を保存
            average_file_path = os.path.join(average_dir_path, "result.txt")
            try:
                with open(average_file_path, 'w') as avg_file:
                    for index, avg in averages.items():
                        avg_file.write(f"Column {index+1} average: {avg:.6f}\n")
                    print(f"Averages saved to {average_file_path}")
            except Exception as e:
                print(f"Failed to write averages to {average_file_path}: {e}")

    except Exception as e:
        print(f"An unexpected error occurred: {e}")


In [66]:
value = "13_14_15_K,N,S,Y,F"
calculate_surf_average(value)

Averages saved to /Users/teraimao/experiment/work/13_14_15_K,N,S,Y,F_work/analysis/ana_gbsw_out/surf_area/average/result.txt


In [67]:
import os
import pandas as pd
import re  # 正規表現を使ってデータを抽出

def update_csv_in(value):
    # エネルギー関連のデータマッピングリスト
    csv_list = ['coul_ener', 'hbond', 'van_der_ener', 'solv_free_ener']
    csv_experiment_data = ['クーロン相互作用エネルギー', '分子内水素結合数', 'ファンデルワールスエネルギー', '溶媒和自由エネルギー']
    
    # 溶媒露出表面積関連のデータマッピング
    surf_map = {
        2: '溶媒露出表面積',
        3: '疎水性残基溶媒露出表面積',
        4: '極性残基溶媒露出表面積',
        5: '荷電残基溶媒露出表面積'
    }

    csv_file = "/Users/teraimao/experiment/data/csv_data/experiment_data_out.csv"

    # 各値を保存する辞書
    results = {}

    # **エネルギー関連の値を取得**
    for j, column_name in zip(csv_list, csv_experiment_data):
        result_file = f"/Users/teraimao/experiment/work/{value}_work/analysis/ana_gbsw_out/{j}/average/result.txt"

        if os.path.isdir(result_file):
            print(f"Error: {result_file} is a directory.")
            continue

        os.makedirs(os.path.dirname(result_file), exist_ok=True)

        if not os.path.exists(result_file):
            with open(result_file, 'w') as file:
                file.write("0.0\n")

        try:
            with open(result_file, 'r') as file:
                file_value = file.readline().strip()
                results[column_name] = float(file_value) if file_value else 0.0
        except Exception as e:
            print(f"Error reading {result_file}: {e}")
            results[column_name] = 0.0

    # **溶媒露出表面積関連の値を取得**
    surf_file = f"/Users/teraimao/experiment/work/{value}_work/analysis/ana_gbsw_out/surf_area/average/result.txt"
    if os.path.exists(surf_file) and not os.path.isdir(surf_file):
        try:
            with open(surf_file, 'r') as file:
                for line in file:
                    match = re.match(r"Column (\d+) average: ([\d\.]+)", line.strip())
                    if match:
                        col_num = int(match.group(1))  # 列番号
                        avg_value = float(match.group(2))  # 平均値
                        if col_num in surf_map:
                            results[surf_map[col_num]] = avg_value
        except Exception as e:
            print(f"Error reading {surf_file}: {e}")
    else:
        print(f"Warning: {surf_file} not found. Initializing values as 0.")
        for col_num in surf_map:
            results[surf_map[col_num]] = 0.0

    # **CSV ファイルの更新**
    try:
        if os.path.exists(csv_file):
            df = pd.read_csv(csv_file)
        else:
            df = pd.DataFrame(columns=['CycPeptMPDB_ID'] + list(results.keys()))
            print(f"Created a new CSV file: {csv_file}")

        for column_name in results.keys():
            if column_name not in df.columns:
                df[column_name] = None

        if value in df['CycPeptMPDB_ID'].values:
            for column_name, result_value in results.items():
                if pd.isna(df.loc[df['CycPeptMPDB_ID'] == value, column_name].iloc[0]):
                    df.loc[df['CycPeptMPDB_ID'] == value, column_name] = result_value
        else:
            new_row = {'CycPeptMPDB_ID': value, **results}
            df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)

        df.to_csv(csv_file, index=False)
        print(f"Updated CSV file saved: {csv_file}")
    except Exception as e:
        print(f"Error updating CSV file: {e}")


In [68]:
value = "13_14_15_K,N,S,Y,F"
update_csv_in(value)

Updated CSV file saved: /Users/teraimao/experiment/data/csv_data/experiment_data_out.csv


In [1]:
# 元のテキストファイルを読み込み、新しいファイルに出力する
input_file = '/Users/teraimao/experiment/work/0_15_30_nK,N,S,Y,F_work/dum_mem.pdb'  # 元のファイル名
output_file = '/Users/teraimao/experiment/work/0_15_30_nK,N,S,Y,F_work/dum_mem_n_.pdb'  # 変更後のファイル名

# ファイルを読み込み、置き換え処理を実行
with open(input_file, 'r') as file:
    lines = file.readlines()

# 置き換え処理
with open(output_file, 'w') as file:
    for line in lines:
        # 行内の ' O ' を ' N ' に置き換える
        modified_line = line.replace(' O ', ' N ')
        file.write(modified_line)

print(f"置き換え完了。結果は {output_file} に保存されました。")


置き換え完了。結果は /Users/teraimao/experiment/work/0_15_30_nK,N,S,Y,F_work/dum_mem_n_.pdb に保存されました。
